In [1]:
%load_ext autoreload
%autoreload 1

import sys
import os
import numpy as np
import pandas as pd
import sqlalchemy as sa
sys.path.append(os.path.abspath(os.path.join('..','..','..')))
from pudl import pudl, ferc1, eia923, settings, constants
from pudl import models, models_ferc1, models_eia923
from pudl import clean_eia923, clean_ferc1, clean_pudl
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
pudl_engine  = pudl.db_connect_pudl()

In [3]:
pt = models.PUDLBase.metadata.tables

plants_eia_select = sa.sql.select([
    pt['generation_fuel_eia923'],
    pt['plants_eia'].c.plant_name,
    pt['plants_eia'].c.plant_id_pudl
]).\
    where(sa.sql.and_(
        pt['generation_fuel_eia923'].c.plant_id == pt['plants_eia'].c.plant_id
    ))
    
plants_df = pd.read_sql(plants_eia_select, pudl_engine)

In [4]:
#Set the datetimeindex
plants_df = plants_df.set_index(pd.DatetimeIndex(plants_df['report_date'])) #sets report_date as index

In [5]:
plants_df_indexed = plants_df.set_index(['plant_id'], append=True)

In [6]:
#groupby plant_id and by year
plants_gb = plants_df.groupby([pd.TimeGrouper(freq='A'), 'plant_id','plant_name','plant_id_pudl'])
#sum net_gen by year by plant and convert to datafram

plants = plants_gb.agg({'fuel_consumed_total_mmbtu' : np.sum,
                        'net_generation_mwh' : np.sum})

In [7]:
plants['heat_rate_btu_per_kwh'] = (plants['fuel_consumed_total_mmbtu']*1000000) / (plants['net_generation_mwh']*1000)

In [8]:
plants

fuel_consumed_total_mmbtu  \
report_date plant_id plant_name                        plant_id_pudl                              
2009-12-31  2        Bankhead Dam                      851                            2758750.0   
            3        Barry                             32                           122482205.0   
            4        Walter Bouldin Dam                850                            8427915.0   
            7        Gadsden                           204                            3886265.0   
            8        Gorgas                            227                           52697640.0   
            9        Copper                            129                             225529.0   
            10       Greene County                     245                           25383733.0   
            11       H Neely Henry Dam                 842                            2609220.0   
            12       Holt Dam                          852                            2113616.0   
            13       Jordan Dam                        853                            4169112.0   
            14       Logan Martin Dam                  846                            4996231.0   
            15       Lay Dam                           843                            7466001.0   
            16       Martin Dam                        844                            4825247.0   
            17       Mitchell Dam                      840                            6372224.0   
            18       Lewis Smith Dam                   849                            4316622.0   
            19       Thurlow Dam                       847                            2977181.0   
            20       Weiss Dam                         841                            2830019.0   
            21       Yates Dam                         845                            1785758.0   
            26       E C Gaston                        176                           79700771.0   
            30       Madelia                           1300                               165.0   
            34       Rollins                           1301                            651196.0   
            38       Millers Ferry                     1302                           3289482.0   
            46       Browns Ferry                      1303                         269988673.0   
            47       Colbert                           1304                          32607544.0   
            48       Guntersville                      1305                           7532350.0   
            49       Wheeler Dam                       1306                          15177083.0   
            50       Widows Creek                      1307                          51648015.0   
            51       Dolet Hills                       1                             51926892.0   
            53       Gantt                             1308                             96323.0   
            54       J K Smith                         1309                           2883466.0   
...                                                                                         ...   
2016-12-31  60351    Bison Solar LLC                   8407                            105351.0   
            60352    Boulder Solar Power, LLC          8416                            100086.0   
            60354    Brady II Wind Energy Center       8446                                 0.0   
            60355    Brady Wind Energy Center          8467                            884093.0   
            60389    RE Barren Ridge 1                 8473                            485884.0   
            60405    Chaves Solar, LLC                 8465                            225408.0   
            60406    Roswell Solar, LLC                8478                            520150.0   
            60413    Bonnybrooke PV                    8454                            114419.0   
           

In [21]:
#Pull in the plants_eia860 table
p8_tbl = models.PUDLBase.metadata.tables['plants_eia860']
p8_select = sa.sql.select([p8_tbl,]) #creates a sql Select object
p8 = pd.read_sql(p8_select, pudl_engine) #converts sql object to pandas dataframe
#Set the datetimeindex
#p8 = p8.set_index(pd.DatetimeIndex(p8['year'])) #sets report_date as index

In [24]:
plants = plants.reset_index()

In [25]:
plants['year'] = pd.DatetimeIndex(plants['report_date']).year

In [26]:
plants.merge(p8,on=['plant_id','year'])

,index,report_date,plant_id,plant_name_x,plant_id_pudl,fuel_consumed_total_mmbtu,net_generation_mwh,heat_rate_btu_per_kwh,year,id,...,longitude,balancing_authority_code,balancing_authority_name,grid_voltage_kv,grid_voltage_2_kv,grid_voltage_3_kv,ash_impoundment,ash_impoundment_lined,ash_impoundment_status,natural_gas_pipeline
0,11346,2011-12-31,2,Bankhead Dam,851,1253705.0,1.290350e+05,9.716007e+03,2011,33,...,NaN,None,None,115.00,NaN,NaN,None,None,None,None
1,11347,2011-12-31,3,Barry,32,102443454.0,1.235423e+07,8.292176e+03,2011,34,...,NaN,None,None,230.00,NaN,NaN,None,None,None,None
2,11348,2011-12-31,4,Walter Bouldin Dam,850,4738434.0,4.876940e+05,9.715998e+03,2011,35,...,NaN,None,None,115.00,NaN,NaN,None,None,None,None
3,11349,2011-12-31,7,Gadsden,204,3632868.0,2.302000e+05,1.578136e+04,2011,36,...,NaN,None,None,115.00,NaN,NaN,None,None,None,None
4,11350,2011-12-31,8,Gorgas,227,50694113.0,4.936430e+06,1.026939e+04,2011,37,...,NaN,None,None,230.00,NaN,NaN,None,None,None,None
5,11351,2011-12-31,9,Copper,129,687879.0,3.899700e+04,1.763928e+04,2011,1111,...,NaN,None,None,69.00,NaN,NaN,None,None,None,None
6,11352,2011-12-31,10,Greene County,245,25131048.0,2.475757e+06,1.015085e+04,2011,38,...,NaN,None,None,230.00,NaN,NaN,None,None,None,None
7,11353,2011-12-31,11,H Neely Henry Dam,842,1453603.0,1.496090e+05,9.716013e+03,2011,39,...,NaN,None,None,115.00,NaN,NaN,None,None,None,None
8,11354,2011-12-31,12,Holt Dam,852,778476.0,8.012300e+04,9.716012e+03,2011,40,...,NaN,None,None,115.00,NaN,NaN,None,None,None,None
9,11355,2011-12-31,13,Jordan Dam,853,2187081.0,2.251010e+05,9.715999e+03,2011,41,...,NaN,None,None,115.00,NaN,NaN,None,None,None,None


In [27]:
# It's a mess down here / not being used -->

In [ ]:
#Pull in the generation_fuel table
gf_tbl = models.PUDLBase.metadata.tables['generation_fuel_eia923']
gf_select = sa.sql.select([gf_tbl,]) #creates a sql Select object
gf = pd.read_sql(gf_select, pudl_engine) #converts sql object to pandas dataframe
#Set the datetimeindex
gf = gf.set_index(pd.DatetimeIndex(gf['report_date'])) #sets report_date as index

In [ ]:
#groupby plant_id and by year
gf_plant_gb = gf.groupby([pd.TimeGrouper(freq='A'), 'plant_id'])
#sum net_gen by year by plant and convert to datafram

gf_plant = gf_plant_gb.agg({'fuel_consumed_total_mmbtu' : np.sum,
                            'net_generation_mwh' : np.sum})

In [ ]:
gf_plant['heat_rate_btu_per_kwh'] = (gf_plant['fuel_consumed_total_mmbtu']*1000000) / (gf_plant['net_generation_mwh']*1000)

In [ ]:
gf_heat_rate = gf_plant['heat_rate_btu_per_kwh'].reset_index()

In [ ]:
gf_plant